In [1]:
# Install Required Packages if you don't have them:
# !pip install python-dotenv joblib anthropic hume-api

# Import Statements
import os
from dotenv import load_dotenv
from joblib import load
from pprint import pprint
import time
import traceback

# Custom Modules (Assuming these are in your project structure)
from src.utils.anthropicwrapper import ClaudeChat, ClaudeChatAssess
from src.utils.humewrapper import HumeSentimentAnalyzer
from src.modules import ConversationVerifier

import json
import os

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:

# --- Load Environment Variables ---
load_dotenv(os.path.join(os.path.dirname(os.getcwd()), ".env"))

# --- Timestamp Input ---
timestamp = 1724629705  # Example timestamp, replace with your input 

# --- Load Data ---
directory = f'data/interviews/{timestamp}/'
chatlog = load(os.path.join(directory, "joblib/conversation.joblib"))



In [3]:
# --- Model Setup --- 
try:
    sentiment_analyser = HumeSentimentAnalyzer(api_key=os.getenv("HUME_API_KEY"))

    sentiment_system_prompt = """You are a skilled emotions analyst provided with a detailed breakdown of sentiment analysis scores from Hume.ai, for a single response in an interview to a question from the interviewer. The scores are split into 3 sections. All numbers are from 0 to 1, linearly scaling, with 1 being a very strong representation of the indicator in question.

    First, Emotions. This contains several human emotions with a numerical value indicating the strength of the corresponding emotion.
    Second, Sentiments. This contains a scale from 1 to 9, each containing a numerical value indicating the magnitude of the sentiment of the topic of the conversation. A negative topic such as murder will have a high value lower in the scale, such as 1 or 2, and a positive topic will have a high value from 0 to 1 higher in the scale such as 8 or 9.
    Third, Toxicity. This contains several toxic representations such as hate, insult, etc, with a value from 0 to 1 for each representation identified in the audio.

    Your job is to provide a concise detailed one sentence breakdown of how the individual was feeling for the particular scores provided. You must be highly objective as your job is to discern whether or not a candidate was exhibiting traits which would or would not be fitting for a successful interview. 
    Model your answer beginning with something along the lines of "For this particular response, the candidate..."""
    
    sentiment_summariser = ClaudeChat("claude-3-5-sonnet-20240620", sentiment_system_prompt)

    evaluation_system_prompt = """You are a highly skilled interviewer currently tasked with reviewing a phone screening interview candidate to decide whether they are to pass on to the next stage of the interview process. There is a high volume of candidates and as such only 47% of candidates should be allowed to pass.
    You will be provided with the following information to aid your decision:
    1. A copy of the job description.
    2. Transcript of the phone interview between the interviewer and candidate.
    3. Sentiment analysis summary to provide you with insight into the tone of the candidate in the call
    4. A summary of any factual inaccuracies the candidate may have made during discussions in the call
    5. A copy of the candidate's CV, which will be provided next.

    You are to evaluate the candidate, primarily on the transcript, and use the additional information provided to identify any potential red-flags. Your response should include a detailed breakdown of why the candidate is chosen to continue onwards to further interviewing. You must end the breakdown with a simple one word response on a new line, "pass" or "fail"."""

    candidate_evaluator = ClaudeChatAssess("claude-3-5-sonnet-20240620", evaluation_system_prompt, "data/cvs/cv-deb.pdf")

except Exception as e:
    print(f"An error occurred during model setup: {e}")
    raise



Error processing PDF: no such file: 'data/cvs/cv-deb.pdf'


In [4]:
# --- Reformat Chatlog ---
def reformat_chatlog(chatlog):
    dropped_context = chatlog[3:]
    outputchatlog = []

    for i in range(0, len(dropped_context), 2):
        if i + 1 < len(dropped_context):
            tempdict = {
                'interviewer': dropped_context[i]['content'],
                'candidate': dropped_context[i+1]['content']
            }
            outputchatlog.append(tempdict)
        else:
            break 
            
    return outputchatlog

# --- Process Sentiments ---


In [5]:
def process_sentiments(chatlog_chat, timestamp):
    filepath = os.path.join('data', 'interviews', str(timestamp), 'audio')
    print(f"Current working directory: {os.getcwd()}")
    print(f"Full audio directory path: {os.path.abspath(filepath)}")

    files = [f for f in os.listdir(filepath) if os.path.isfile(os.path.join(filepath, f))]
    
    if len(chatlog_chat) < len(files):
        files = files[:len(chatlog_chat)]

    for f in files:
        print(f"File found: {f}")

    sentiments = []
    
    for count, file in enumerate(files, 1):
        full_file_path = os.path.join(filepath, file)
        print(f"Processing file: {full_file_path}")
        
        # Add a small delay and re-check file existence
        time.sleep(0.1)
        if not os.path.exists(full_file_path):
            print(f"File not found (after delay): {full_file_path}")
            continue

        try:
            result = sentiment_analyser.analyze_audio(full_file_path)
            sentiment_summary = sentiment_summariser.chat(str(result))
            sentiments.append((result, sentiment_summary))
            chatlog_chat[count-1]['sentiment'] = sentiment_summary
        except Exception as e:
            print(f"Error processing file {full_file_path}: {str(e)}")
            traceback.print_exc()

    return chatlog_chat



In [6]:
# --- Evaluate Candidate --- 
def evaluate_candidate(chatlog_chat):
    ConversationVerifier.process_qa_pair(chatlog_chat)
    print("The Feedback JSON from the sentiment analyser and accuracy verifier: \n")
    pprint(chatlog_chat)
    evaluation = candidate_evaluator.chat(str(chatlog_chat))
    return evaluation



In [7]:
# --- Main Execution Flow ---
for i in range(9):
    print(f'---------------------------{timestamp}-{i}------------------------')
    chatlog_chat = reformat_chatlog(chatlog)  # Process the loaded chatlog

    chatlog_chat = process_sentiments(chatlog_chat, timestamp) # Analyze sentiments

    #chatlog_chat_copy = chatlog_chat[1]
    evaluation = evaluate_candidate(chatlog_chat) # Get the final evaluation

    
    chatlog_file_path = f"data/interviews/{timestamp}-{i}/outcome/"

    if not os.path.exists(chatlog_file_path):
        os.makedirs(chatlog_file_path)

    with open(chatlog_file_path+"chatlog.json", "w") as file:
        json.dump(chatlog_chat, file, indent=4)
    evaluation_file_path = f"data/interviews/{timestamp}-{i}/outcome/"
    if not os.path.exists(chatlog_file_path):
        os.makedirs(chatlog_file_path)
    with open(evaluation_file_path+"evaluation.txt", "w") as file:
        file.write(str(evaluation).replace("\\n", "\n"))
    print(f'---------------------------{timestamp}-{i}------------------------')


---------------------------1724629705-0------------------------
Current working directory: d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM
Full audio directory path: d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM\data\interviews\1724629705\audio
File found: audio_1_1724629705.wav
File found: audio_2_1724629705.wav
File found: audio_3_1724629705.wav
File found: audio_4_1724629705.wav
Processing file: data\interviews\1724629705\audio\audio_1_1724629705.wav
Analyzing audio file: data\interviews\1724629705\audio\audio_1_1724629705.wav
File exists: True
Job submitted successfully
Job completed
[{'source': {'type': 'file', 'filename': 'audio_1_1724629705.wav', 'content_type': 'audio/wav', 'md5sum': '0bff7efa6ac3802fbe9bf25f5b4220c7'}, 'results': {'predictions': [{'file': 'audio_1_1724629705.wav', 'file_type': 'audio', 'models': {'language': {'metadata': {'confidence': 0.68603516, 'detected_language': 'fr'}, 'grouped_predictions': [{'id': 'unknown', 'predictions': [{'te

Traceback (most recent call last):
  File "C:\Users\debo1\AppData\Local\Temp\ipykernel_31892\1867170296.py", line 27, in process_sentiments
    result = sentiment_analyser.analyze_audio(full_file_path)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM\src\utils\humewrapper.py", line 77, in analyze_audio
    return self._process_predictions(predictions)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM\src\utils\humewrapper.py", line 94, in _process_predictions
    raise Exception("Failed to process predictions")
Exception: Failed to process predictions


Analyzing audio file: data\interviews\1724629705\audio\audio_2_1724629705.wav
File exists: True
Job submitted successfully
Job completed
[{'source': {'type': 'file', 'filename': 'audio_2_1724629705.wav', 'content_type': 'audio/wav', 'md5sum': '70a9526b605ea35c351a038ec2156ba8'}, 'results': {'predictions': [{'file': 'audio_2_1724629705.wav', 'file_type': 'audio', 'models': {'language': {'metadata': {'confidence': 0.98831725, 'detected_language': 'en'}, 'grouped_predictions': [{'id': 'unknown', 'predictions': [{'text': "I'm sure. So I'm studying artificial intelligence of the university of Kent currently and for my final dis edition. I'm working on making a automated screening interview agent. To implement this, I have used a rack pipeline, Mainly as the accuracy verifies verify. So what happens is when the candidate answers their questions. It goes through two pipelines. One is the sentiment analysis and one is the accuracy verify. For the accuracy verify, I have implemented our retriev

Traceback (most recent call last):
  File "C:\Users\debo1\AppData\Local\Temp\ipykernel_31892\1867170296.py", line 27, in process_sentiments
    result = sentiment_analyser.analyze_audio(full_file_path)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM\src\utils\humewrapper.py", line 77, in analyze_audio
    return self._process_predictions(predictions)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM\src\utils\humewrapper.py", line 94, in _process_predictions
    raise Exception("Failed to process predictions")
Exception: Failed to process predictions


Analyzing audio file: data\interviews\1724629705\audio\audio_3_1724629705.wav
File exists: True
Job submitted successfully
Job completed
[{'source': {'type': 'file', 'filename': 'audio_3_1724629705.wav', 'content_type': 'audio/wav', 'md5sum': '5705a3916e86facf2b6202b9fa12c165'}, 'results': {'predictions': [{'file': 'audio_3_1724629705.wav', 'file_type': 'audio', 'models': {'language': {'metadata': {'confidence': 0.9893276, 'detected_language': 'en'}, 'grouped_predictions': [{'id': 'unknown', 'predictions': [{'text': "Yes. So to improve the context of the retrieval quality of the rag pipeline, I had to break down The answer from the candidate and the searchable strings with the help of an. So let's say an answer can be broken down into six query strings. Each of the six query strings would then go on to... Each of... Sorry. Each of these six query strings would then be used to search in Google, and we would draw the context from the first two web pages. So in a total, we would get the i

...

Analyzing audio file: data\interviews\1724629705\audio\audio_3_1724629705.wav
File exists: True
Job submitted successfully
Job completed
[{'source': {'type': 'file', 'filename': 'audio_3_1724629705.wav', 'content_type': 'audio/wav', 'md5sum': '5705a3916e86facf2b6202b9fa12c165'}, 'results': {'predictions': [{'file': 'audio_3_1724629705.wav', 'file_type': 'audio', 'models': {'language': {'metadata': {'confidence': 0.98819315, 'detected_language': 'en'}, 'grouped_predictions': [{'id': 'unknown', 'predictions': [{'text': "Yes. So to improve the context of the retrieval quality of the rag pipeline, I had to break down The answer from the candidate and the searchable strings with the help of an. So let's say an answer can be broken down into six query strings. Each of the six query strings would then go on to... Each of... Sorry. Each of these six query strings would then be used to search in Google, and we would draw the context from the first two web pages. So in a total, we would get the 

Traceback (most recent call last):
  File "C:\Users\debo1\AppData\Local\Temp\ipykernel_31892\1867170296.py", line 27, in process_sentiments
    result = sentiment_analyser.analyze_audio(full_file_path)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM\src\utils\humewrapper.py", line 77, in analyze_audio
    return self._process_predictions(predictions)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM\src\utils\humewrapper.py", line 94, in _process_predictions
    raise Exception("Failed to process predictions")
Exception: Failed to process predictions


Analyzing audio file: data\interviews\1724629705\audio\audio_4_1724629705.wav
File exists: True
Job submitted successfully
Job completed
[{'source': {'type': 'file', 'filename': 'audio_4_1724629705.wav', 'content_type': 'audio/wav', 'md5sum': '3475dd174cc4b9dc225984655c1d2eb2'}, 'results': {'predictions': [{'file': 'audio_4_1724629705.wav', 'file_type': 'audio', 'models': {'language': {'metadata': {'confidence': 0.9888263, 'detected_language': 'en'}, 'grouped_predictions': [{'id': 'unknown', 'predictions': [{'text': 'For model selection, we choose Gb four opinion, mainly because we use lan to implement the right pipeline and Gp four o mini, had the perfect balance of intelligence and cost effectiveness. And also speed that we had to manage, and this was just to verify the answers. So we did not go for a most sophisticated model such as claude on it, three point five which by all... Which considered the most intelligent element l till now. We did not need such a a high powered L. We jus

Traceback (most recent call last):
  File "C:\Users\debo1\AppData\Local\Temp\ipykernel_31892\1867170296.py", line 27, in process_sentiments
    result = sentiment_analyser.analyze_audio(full_file_path)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM\src\utils\humewrapper.py", line 77, in analyze_audio
    return self._process_predictions(predictions)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Kent\University Of Kent UK\Projects\Disso\Screening-LLM\src\utils\humewrapper.py", line 94, in _process_predictions
    raise Exception("Failed to process predictions")
Exception: Failed to process predictions


attempting to generate searc queries from answers
search queries generated for answer to question: Hello! Thank you for taking the time to speak with me today about the Entry-Level RAG AI Engineer role. I'd like to start by asking you a few questions about your experience and skills. Could you tell me about any projects you've worked on involving retrieval-augmented generation (RAG) pipelines?

queries are: ['1. "Retrieval-augmented generation (RAG) pipeline in automated screening interview agent"', '2. "Implementing retrieval-augmented generation for accuracy verification in AI projects"']


search queries generated for answer to question: That's an interesting project. Can you elaborate on the specific challenges you faced while implementing the RAG pipeline for your accuracy verifier? How did you address issues like retrieval quality or context relevance?

queries are: ['1. "Improving context and retrieval quality in RAG pipeline"', '2. "Query Decomposition technique for accuracy ve